<a href="https://colab.research.google.com/github/FernandaSzAdr/KNN_cromossome/blob/master/TesteRedes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers.core import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
train_path = 'drive/My Drive/base_crcn/Base/Train'
test_path = 'drive/My Drive/base_crcn/Base/Test'



---



## Implementação do código descrito em **"Crowdsourcing for Chromosome Segmentation and Deep Classification"**

In [0]:
bSize = 32

In [0]:
datagen = ImageDataGenerator(rescale=1/255.)

In [7]:
train_batches = datagen.flow_from_directory(train_path, target_size=(120, 120), classes=['C', 'NC'], batch_size=bSize)
test_batches = datagen.flow_from_directory(test_path, target_size=(120, 120), classes=['C', 'NC'], batch_size=bSize)

Found 5344 images belonging to 2 classes.
Found 1336 images belonging to 2 classes.


In [8]:
test_batches.class_indices

{'C': 0, 'NC': 1}



---



### 1 tentativa


In [0]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(120, 120, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(32, (3, 3), activation='relu'),
    Conv2D(32, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(16, (3, 3), activation='relu'),
    Conv2D(16, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(8, (3, 3), activation='relu'),
    Conv2D(8, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(1024, activation='sigmoid'),
    Dense(512, activation='sigmoid'),
   
    Dense(2, activation='softmax')
]
)

In [0]:
model.summary()

In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
stepsPerEpoche = len(train_batches)/bSize

In [21]:
model.fit_generator(train_batches, steps_per_epoch=stepsPerEpoche, epochs=50, verbose=2)

Epoch 1/50
 - 1s - loss: 0.7173 - acc: 0.4740
Epoch 2/50
 - 1s - loss: 0.6992 - acc: 0.5208
Epoch 3/50
 - 1s - loss: 0.6939 - acc: 0.4896
Epoch 4/50
 - 1s - loss: 0.6954 - acc: 0.4896
Epoch 5/50
 - 1s - loss: 0.7159 - acc: 0.5677
Epoch 6/50
 - 1s - loss: 0.7185 - acc: 0.5156
Epoch 7/50
 - 8s - loss: 0.7071 - acc: 0.4948
Epoch 8/50
 - 9s - loss: 0.6952 - acc: 0.5104
Epoch 9/50
 - 1s - loss: 0.7090 - acc: 0.4948
Epoch 10/50
 - 1s - loss: 0.7067 - acc: 0.4740
Epoch 11/50
 - 1s - loss: 0.6985 - acc: 0.4688
Epoch 12/50
 - 1s - loss: 0.7077 - acc: 0.4740
Epoch 13/50
 - 8s - loss: 0.6971 - acc: 0.5156
Epoch 14/50
 - 9s - loss: 0.7062 - acc: 0.4688
Epoch 15/50
 - 1s - loss: 0.7003 - acc: 0.5260
Epoch 16/50
 - 1s - loss: 0.6913 - acc: 0.5469
Epoch 17/50
 - 1s - loss: 0.6959 - acc: 0.5052
Epoch 18/50
 - 9s - loss: 0.7211 - acc: 0.4948
Epoch 19/50
 - 8s - loss: 0.7078 - acc: 0.4740
Epoch 20/50
 - 1s - loss: 0.7006 - acc: 0.5000
Epoch 21/50
 - 10s - loss: 0.7003 - acc: 0.4531
Epoch 22/50
 - 1s - l

In [0]:
model.save_weights('50_epochs_exemplo1.h5')



---



### 2 tentativa

In [0]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(120, 120, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(32, (3, 3), activation='relu'),
    Conv2D(32, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(16, (3, 3), activation='relu'),
    Conv2D(16, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(8, (3, 3), activation='relu'),
    Conv2D(8, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
   
    Dense(2, activation='softmax')
]
)

In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
stepsPerEpoche = len(train_batches)/bSize

In [27]:
model.fit_generator(train_batches, steps_per_epoch=stepsPerEpoche, epochs=50, verbose=2)

Epoch 1/50
 - 2s - loss: 0.6931 - acc: 0.4948
Epoch 2/50
 - 1s - loss: 0.6932 - acc: 0.4896
Epoch 3/50
 - 1s - loss: 0.6932 - acc: 0.4792
Epoch 4/50
 - 1s - loss: 0.6932 - acc: 0.5312
Epoch 5/50
 - 1s - loss: 0.6929 - acc: 0.5052
Epoch 6/50
 - 1s - loss: 0.6928 - acc: 0.5208
Epoch 7/50
 - 1s - loss: 0.6924 - acc: 0.5104
Epoch 8/50
 - 1s - loss: 0.6926 - acc: 0.5990
Epoch 9/50
 - 1s - loss: 0.6923 - acc: 0.6302
Epoch 10/50
 - 1s - loss: 0.6918 - acc: 0.5729
Epoch 11/50
 - 1s - loss: 0.6919 - acc: 0.5000
Epoch 12/50
 - 1s - loss: 0.6916 - acc: 0.5365
Epoch 13/50
 - 1s - loss: 0.6905 - acc: 0.7500
Epoch 14/50
 - 1s - loss: 0.6894 - acc: 0.5365
Epoch 15/50
 - 1s - loss: 0.6884 - acc: 0.5625
Epoch 16/50
 - 1s - loss: 0.6891 - acc: 0.4844
Epoch 17/50
 - 1s - loss: 0.6862 - acc: 0.5781
Epoch 18/50
 - 1s - loss: 0.6909 - acc: 0.4844
Epoch 19/50
 - 1s - loss: 0.6891 - acc: 0.5260
Epoch 20/50
 - 1s - loss: 0.6888 - acc: 0.6510
Epoch 21/50
 - 1s - loss: 0.6846 - acc: 0.7500
Epoch 22/50
 - 1s - lo

In [0]:
model.save_weights('50_epochs_exemplo1_2.h5')

### 3 tentativa

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, X_train.shape[3])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(1e-4)))

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Implementação código do https://medium.com/@ksusorokina/image-classification-with-convolutional-neural-networks-496815db12a8

In [0]:
model = Sequential()

In [0]:
model.add(Conv2D(32, (3, 3), input_shape=(120, 120, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



---



In [0]:
bSize = 16

In [0]:
datagen = ImageDataGenerator(rescale=1/255.)

In [9]:
train_batches = datagen.flow_from_directory(train_path, target_size=(120, 120), classes=['C', 'NC'], batch_size=bSize)
test_batches = datagen.flow_from_directory(test_path, target_size=(120, 120), classes=['C', 'NC'], batch_size=bSize)

Found 5344 images belonging to 2 classes.
Found 1336 images belonging to 2 classes.


In [0]:
stepsPerEpoche = len(train_batches)/bSize

In [11]:
model.fit_generator(train_batches, steps_per_epoch=stepsPerEpoche, 
                    epochs=25, verbose=2)


Epoch 1/25
 - 175s - loss: 0.7446 - acc: 0.5357
Epoch 2/25
 - 165s - loss: 0.6525 - acc: 0.6473
Epoch 3/25
 - 171s - loss: 0.5413 - acc: 0.7634
Epoch 4/25
 - 163s - loss: 0.4807 - acc: 0.8185
Epoch 5/25
 - 164s - loss: 0.3512 - acc: 0.8661
Epoch 6/25
 - 154s - loss: 0.4099 - acc: 0.8482
Epoch 7/25
 - 159s - loss: 0.3730 - acc: 0.8676
Epoch 8/25
 - 156s - loss: 0.3657 - acc: 0.8527
Epoch 9/25
 - 167s - loss: 0.3642 - acc: 0.8646
Epoch 10/25
 - 157s - loss: 0.3688 - acc: 0.8631
Epoch 11/25
 - 158s - loss: 0.2333 - acc: 0.9196
Epoch 12/25
 - 165s - loss: 0.2734 - acc: 0.8810
Epoch 13/25
 - 163s - loss: 0.3091 - acc: 0.9003
Epoch 14/25
 - 161s - loss: 0.3166 - acc: 0.8973
Epoch 15/25
 - 179s - loss: 0.2324 - acc: 0.9226
Epoch 16/25
 - 149s - loss: 0.3429 - acc: 0.9018
Epoch 17/25
 - 1s - loss: 0.2660 - acc: 0.9003
Epoch 18/25
 - 1s - loss: 0.2177 - acc: 0.9226
Epoch 19/25
 - 1s - loss: 0.2947 - acc: 0.9033
Epoch 20/25
 - 1s - loss: 0.1697 - acc: 0.9360
Epoch 21/25
 - 1s - loss: 0.2140 - ac

In [0]:
model.save_weights('25_epochs_exemplo2.h5')

In [11]:
model.fit_generator(train_batches, steps_per_epoch=stepsPerEpoche, 
                    epochs=50, verbose=2)

Epoch 1/50
 - 2s - loss: 0.7353 - acc: 0.4940
Epoch 2/50
 - 1s - loss: 0.6482 - acc: 0.6443
Epoch 3/50
 - 1s - loss: 0.5639 - acc: 0.7693
Epoch 4/50
 - 1s - loss: 0.5024 - acc: 0.7723
Epoch 5/50
 - 1s - loss: 0.4525 - acc: 0.7812
Epoch 6/50
 - 1s - loss: 0.3668 - acc: 0.8631
Epoch 7/50
 - 1s - loss: 0.3639 - acc: 0.8646
Epoch 8/50
 - 1s - loss: 0.3626 - acc: 0.8586
Epoch 9/50
 - 1s - loss: 0.3493 - acc: 0.8527
Epoch 10/50
 - 1s - loss: 0.2751 - acc: 0.8958
Epoch 11/50
 - 1s - loss: 0.4273 - acc: 0.8646
Epoch 12/50
 - 1s - loss: 0.3008 - acc: 0.9241
Epoch 13/50
 - 1s - loss: 0.3326 - acc: 0.8795
Epoch 14/50
 - 1s - loss: 0.2798 - acc: 0.8824
Epoch 15/50
 - 1s - loss: 0.3081 - acc: 0.8899
Epoch 16/50
 - 1s - loss: 0.3306 - acc: 0.8914
Epoch 17/50
 - 1s - loss: 0.3057 - acc: 0.8810
Epoch 18/50
 - 1s - loss: 0.2491 - acc: 0.9226
Epoch 19/50
 - 1s - loss: 0.2335 - acc: 0.9301
Epoch 20/50
 - 1s - loss: 0.3320 - acc: 0.9077
Epoch 21/50
 - 1s - loss: 0.2576 - acc: 0.9167
Epoch 22/50
 - 1s - lo

In [0]:
model.save_weights('50_epochs_exemplo2.h5')